## Example of Image-to-Recipe Pipeline

In [ ]:
import os
from src.keras_veggies import *
from src.recipeScripts import *
from src.miscScripts import *

### Load in the dataset

In [ ]:
# import file recipe dataset
with open('../data/processed/df_epi_cleaner-5.pkl','rb') as fin:
    df = pickle.load(fin)

### Calling functions, feature extractor and classifier

In [ ]:
# Calling Feature-Extractor and Predictor

extractor = callModel('monet')
predictor = load_model('models/trained_model.hdf5')
class_dictionary = np.load('models/class_indices.npy').item()
inv_map = {v:k for k,v in class_dictionary.items()}

In [ ]:
# Specify directory of test data (images)
image_path_dir = '../data/images/testData/'

print('... models loaded!')

# Loop over each file
result=[]
for pic_file in os.listdir(image_path_dir):
    
    image = load_img(image_path_dir+pic_file, target_size=(224,224))
    image = img_to_array(image) / 255
    image = np.expand_dims(image, axis=0)
    
    # extractor is called above 
    features = extractor.predict(image)
    # predictor
    class_predicted = predictor.predict_classes(features)
    probabilities = predictor.predict_proba(features)
    
    inID = class_predicted[0]
    label = inv_map[inID]
    
    result.append(label)
    
print('The images are: ', ', '.join(result))

# Find recipes with intersections
d3, d2, d1, d = intersect(df, result)
# Output recipes
out3, links, images =outputRecipes(d3)
out2, links, images =outputRecipes(d2)
out1, links, images =outputRecipes(d1)

print('\n')
print('Here are the top recipes for all 3 ingredients: ')
print(out3.title.head(3),'\n')

print('Here are the top recipes for 2 of 3 ingredients: ')
print(out2.title.head(3),'\n')

print('Here are the top recipes for (at least) 1 of 3 ingredients: ')
print(out1.title.head(3))


In [ ]:
# Checkout the photos of ingredients, if available
# Show picture
showcasePhotos(d2.head(2))